In [1]:
###################importing libraries##############################
from __future__ import print_function
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from openpyxl import load_workbook
from tensorflow.keras.layers import LayerNormalization
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import load_model
from pandas import ExcelWriter
from keras.layers import GaussianNoise
import statistics


Using TensorFlow backend.


In [ ]:
####################Reading the excel Files####################################
path='G:\\Class_project\\'
x_train=pd.read_excel(path+'B_train_samples.xlsx')
y_train=pd.read_excel(path+'B_train_labels.xlsx')
x_test=pd.read_excel(path+'B_test_samples.xlsx')
y_test=pd.read_excel(path+'B_test_labels.xlsx')
x_train=x_train[['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21',
       'F22']]
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
type(y_train)

In [ ]:
####################scaling the training Data######################################
scaler = MinMaxScaler()
x_train_scl = scaler.fit_transform(x_train)
x_test_scl = scaler.transform(x_test)
#x_train_scl=to_dataframe(x_train_scl)
x_train = pd.DataFrame(data=x_train_scl)
x_test=pd.DataFrame(data=x_test_scl)

In [ ]:
################Creating Function to fit and evaluate the data###################
def fit_and_evaluate(b_t_x, b_val_x, b_t_y, b_val_y, callbacks,EPOCHS=100, BATCH_SIZE=1000):
    results = model.fit(b_t_x, b_t_y,callbacks=callbacks, epochs=EPOCHS, batch_size=BATCH_SIZE, 
              verbose=1, validation_split=0.1) 
    saved_model = load_model('b_best_model.h5')
    print("Val Score: ", saved_model.evaluate(b_val_x, b_val_y))
    return results

In [ ]:
accuracy = []
f_score = []
y_pred_list = []
t=y_train.columns
count=0
#####running the model 10 time###########
for j in range(10):
    for i in range(y_train.shape[1]):
        temp = ""
        temp1 = ""
        y_train_1=y_train[t[i]]
        y_train_1=pd.DataFrame(y_train_1)
        
        ##########creating the model#####################
        model = Sequential()
        model.add(Dense(64, activation='relu', input_dim=42))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.1)))
        model.add(BatchNormalization())
        model.add(Dense(12, activation='softmax'))
        model.add(GaussianNoise(0.1))
        model.add(Dense(512, activation = 'linear'))
        
        model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=10)
        mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
    
    
        model_history = [] 
        t_x, val_x, t_y, val_y = train_test_split(x_train, y_train_1, test_size=0.3)
        
        ############training and evaluating the model###################
        model_history.append(fit_and_evaluate(t_x, val_x, t_y, val_y,[es,mc], 100,1000))
        saved_model = load_model('best_model.h5')
        score = saved_model.evaluate(x_test,y_test,batch_size=1000)
        print('score:',score)
        
        #####accuracy#########
        temp = str(round(score[1],2))
        accuracy.append(temp)
        print('accuracy:',accuracy)
        
        ########predicting values#####################
        y_pred=saved_model.predict_classes(x_test)
        y_pred_list.append(y_pred)
        
        ##########f_score######################
        f1= f1_score(y_test, y_pred,average='micro')
        print(f1)
        temp1 = str(round(f1,2))
        print(temp1)
        f_score.append(temp1)
    

print("accuracy")    
print(accuracy)
print("f_score")
print(f_score)

########printing mean and standard deviation#########################
print(accuracy[i::i+25].mean() for i in range(25))
        

print(f_score[i::i+25].std() for i in range(25))


In [ ]:

###########writing values to excel files########################

writer = pd.ExcelWriter('predictions_b.xlsx', engine='xlsxwriter')
df1 = pd.DataFrame({'B': y_pred_list[0], 'C5': y_pred_list[1], 'C10': y_pred_list[2], 'C15': y_pred_list[3], 'C20': y_pred_list[4], 'C25': y_pred_list[5], 'C30': y_pred_list[6], 'C35': y_pred_list[7], 'C40': y_pred_list[8], 'A5': y_pred_list[9], 'A10': y_pred_list[10], 'A15': y_pred_list[11], 'A20': y_pred_list[12], 'A25': y_pred_list[13], 'A30': y_pred_list[14], 'A35': y_pred_list[15], 'A40': y_pred_list[16], 'N5': y_pred_list[17], 'N1': y_pred_list[18], 'N15': y_pred_list[19], 'N20': y_pred_list[2], 'N25': y_pred_list[21], 'N30': y_pred_list[22], 'N35': y_pred_list[23], 'N40': y_pred_list[24]})
df1.to_excel(writer, sheet_name='Sheet_name_1')
writer.save()
        
writer = pd.ExcelWriter('predictions_b.xlsx', engine='openpyxl')
writer.book = load_workbook('predictions_b.xlsx')
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
reader = pd.read_excel('predictions_b.xlsx')
df2 = pd.DataFrame({'B': y_pred_list[25], 'C5': y_pred_list[26], 'C10': y_pred_list[27], 'C15': y_pred_list[28], 'C20': y_pred_list[29], 'C25': y_pred_list[30], 'C30': y_pred_list[31], 'C35': y_pred_list[32], 'C40': y_pred_list[33], 'A5': y_pred_list[34], 'A10': y_pred_list[35], 'A15': y_pred_list[36], 'A20': y_pred_list[37], 'A25': y_pred_list[38], 'A30': y_pred_list[39], 'A35': y_pred_list[40], 'A40': y_pred_list[41], 'N5': y_pred_list[42], 'N10': y_pred_list[43], 'N15': y_pred_list[44], 'N20': y_pred_list[45], 'N25': y_pred_list[46], 'N30': y_pred_list[47], 'N35': y_pred_list[48], 'N40': y_pred_list[49]})
df2.to_excel(writer, sheet_name='Sheet_name_2')
writer.save()

writer = pd.ExcelWriter('predictions_b.xlsx', engine='openpyxl')
writer.book = load_workbook('predictions_b.xlsx')
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
reader = pd.read_excel(r'predictions_b.xlsx')
df3 = pd.DataFrame({'B': y_pred_list[50], 'C5': y_pred_list[51], 'C10': y_pred_list[52], 'C15': y_pred_list[53], 'C20': y_pred_list[54], 'C25': y_pred_list[55], 'C30': y_pred_list[56], 'C35': y_pred_list[57], 'C40': y_pred_list[58], 'A5': y_pred_list[59], 'A10': y_pred_list[60], 'A15': y_pred_list[61], 'A20': y_pred_list[62], 'A25': y_pred_list[63], 'A30': y_pred_list[64], 'A35': y_pred_list[65], 'A40': y_pred_list[66], 'N5': y_pred_list[67], 'N10': y_pred_list[68], 'N15': y_pred_list[69], 'N20': y_pred_list[70], 'N25': y_pred_list[71], 'N30': y_pred_list[72], 'N35': y_pred_list[73], 'N40': y_pred_list[74]})
df3.to_excel(writer, sheet_name='Sheet_name_3')
writer.save()

writer = pd.ExcelWriter('predictions_b.xlsx', engine='openpyxl')
writer.book = load_workbook('predictions_b.xlsx')
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
reader = pd.read_excel(r'predictions_b.xlsx')
df4 = pd.DataFrame({'B': y_pred_list[75], 'C5': y_pred_list[76], 'C13': y_pred_list[77], 'C15': y_pred_list[78], 'C20': y_pred_list[79], 'C25': y_pred_list[80], 'C33': y_pred_list[81], 'C35': y_pred_list[82], 'C43': y_pred_list[83], 'A5': y_pred_list[84], 'A13': y_pred_list[85], 'A15': y_pred_list[86], 'A23': y_pred_list[87], 'A25': y_pred_list[88], 'A33': y_pred_list[89], 'A35': y_pred_list[90], 'A43': y_pred_list[91], 'N5': y_pred_list[92], 'N13': y_pred_list[93], 'N15': y_pred_list[94], 'N23': y_pred_list[95], 'N25': y_pred_list[96], 'N33': y_pred_list[97], 'N35': y_pred_list[98], 'N40': y_pred_list[99]})
df4.to_excel(writer,sheet_name='Sheet_name_4')
writer.save()

writer = pd.ExcelWriter('predictions_b.xlsx', engine='openpyxl')
writer.book = load_workbook('predictions_b.xlsx')
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
reader = pd.read_excel(r'predictions_b.xlsx')
df5 = pd.DataFrame({'B': y_pred_list[100], 'C5': y_pred_list[101], 'C14': y_pred_list[102], 'C15': y_pred_list[103], 'C24': y_pred_list[104], 'C25': y_pred_list[105], 'C34': y_pred_list[106], 'C35': y_pred_list[107], 'C44': y_pred_list[108], 'A5': y_pred_list[109], 'A14': y_pred_list[110], 'A15': y_pred_list[111], 'A24': y_pred_list[112], 'A25': y_pred_list[113], 'A34': y_pred_list[114], 'A35': y_pred_list[115], 'A44': y_pred_list[116], 'N5': y_pred_list[117], 'N14': y_pred_list[118], 'N15': y_pred_list[119], 'N24': y_pred_list[120], 'N25': y_pred_list[121], 'N34': y_pred_list[122], 'N35': y_pred_list[123], 'N40': y_pred_list[124]})
df5.to_excel(writer,sheet_name='Sheet_name_5')
writer.save()

writer = pd.ExcelWriter('predictions_b.xlsx', engine='openpyxl')
writer.book = load_workbook('predictions_b.xlsx')
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
reader = pd.read_excel(r'predictions_b.xlsx')
df6 = pd.DataFrame({'B': y_pred_list[125], 'C5': y_pred_list[126], 'C15': y_pred_list[127], 'C15': y_pred_list[128], 'C25': y_pred_list[129], 'C25': y_pred_list[130], 'C35': y_pred_list[131], 'C35': y_pred_list[132], 'C45': y_pred_list[133], 'A5': y_pred_list[134], 'A15': y_pred_list[135], 'A15': y_pred_list[136], 'A25': y_pred_list[137], 'A25': y_pred_list[138], 'A35': y_pred_list[139], 'A35': y_pred_list[140], 'A45': y_pred_list[141], 'N5': y_pred_list[142], 'N15': y_pred_list[143], 'N15': y_pred_list[144], 'N25': y_pred_list[145], 'N25': y_pred_list[146], 'N35': y_pred_list[147], 'N35': y_pred_list[148], 'N45': y_pred_list[149]})
df6.to_excel(writer,sheet_name='Sheet_name_6')
writer.save()

writer = pd.ExcelWriter('predictions_b.xlsx', engine='openpyxl')
writer.book = load_workbook('predictions_b.xlsx')
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
reader = pd.read_excel(r'predictions_b.xlsx')
df7 = pd.DataFrame({'B': y_pred_list[150], 'C5': y_pred_list[151], 'C16': y_pred_list[152], 'C15': y_pred_list[153], 'C26': y_pred_list[154], 'C25': y_pred_list[155], 'C36': y_pred_list[156], 'C35': y_pred_list[157], 'C46': y_pred_list[158], 'A5': y_pred_list[159], 'A16': y_pred_list[160], 'A15': y_pred_list[161], 'A26': y_pred_list[162], 'A25': y_pred_list[163], 'A36': y_pred_list[164], 'A35': y_pred_list[165], 'A46': y_pred_list[166], 'N5': y_pred_list[167], 'N16': y_pred_list[168], 'N15': y_pred_list[169], 'N26': y_pred_list[170], 'N25': y_pred_list[171], 'N36': y_pred_list[172], 'N35': y_pred_list[173], 'N46': y_pred_list[174]})
df7.to_excel(writer,sheet_name='Sheet_name_7')
writer.save()

writer = pd.ExcelWriter('predictions_b.xlsx', engine='openpyxl')
writer.book = load_workbook('predictions_b.xlsx')
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
reader = pd.read_excel(r'predictions_b.xlsx')
df8 = pd.DataFrame({'B': y_pred_list[175], 'C5': y_pred_list[176], 'C17': y_pred_list[177], 'C15': y_pred_list[178], 'C27': y_pred_list[179], 'C25': y_pred_list[180], 'C37': y_pred_list[181], 'C35': y_pred_list[182], 'C47': y_pred_list[183], 'A5': y_pred_list[184], 'A17': y_pred_list[185], 'A15': y_pred_list[186], 'A27': y_pred_list[187], 'A25': y_pred_list[188], 'A37': y_pred_list[189], 'A35': y_pred_list[190], 'A47': y_pred_list[191], 'N5': y_pred_list[192], 'N17': y_pred_list[193], 'N15': y_pred_list[194], 'N27': y_pred_list[195], 'N25': y_pred_list[196], 'N37': y_pred_list[197], 'N35': y_pred_list[198], 'N47': y_pred_list[199]})
df8.to_excel(writer,sheet_name='Sheet_name_8')
writer.save()

writer = pd.ExcelWriter('predictions_b.xlsx', engine='openpyxl')
writer.book = load_workbook('predictions_b.xlsx')
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
reader = pd.read_excel(r'predictions_b.xlsx')
df9 = pd.DataFrame({'B': y_pred_list[200], 'C5': y_pred_list[201], 'C18': y_pred_list[202], 'C15': y_pred_list[203], 'C28': y_pred_list[204], 'C25': y_pred_list[205], 'C38': y_pred_list[206], 'C35': y_pred_list[207], 'C48': y_pred_list[208], 'A5': y_pred_list[209], 'A18': y_pred_list[210], 'A15': y_pred_list[211], 'A28': y_pred_list[212], 'A25': y_pred_list[213], 'A38': y_pred_list[214], 'A35': y_pred_list[215], 'A48': y_pred_list[216], 'N5': y_pred_list[217], 'N18': y_pred_list[218], 'N15': y_pred_list[219], 'N28': y_pred_list[220], 'N25': y_pred_list[221], 'N38': y_pred_list[222], 'N35': y_pred_list[223], 'N48': y_pred_list[224]})
df9.to_excel(writer,sheet_name='Sheet_name_9')
writer.save()


writer = pd.ExcelWriter('predictions_b.xlsx', engine='openpyxl')
writer.book = load_workbook('predictions_b.xlsx')
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
reader = pd.read_excel(r'predictions_b.xlsx')
df10 = pd.DataFrame({'B': y_pred_list[225], 'C5': y_pred_list[226], 'C19': y_pred_list[227], 'C15': y_pred_list[228], 'C29': y_pred_list[229], 'C25': y_pred_list[230], 'C39': y_pred_list[231], 'C35': y_pred_list[232], 'C49': y_pred_list[233], 'A5': y_pred_list[234], 'A19': y_pred_list[235], 'A15': y_pred_list[236], 'A29': y_pred_list[237], 'A25': y_pred_list[238], 'A39': y_pred_list[239], 'A35': y_pred_list[240], 'A49': y_pred_list[241], 'N5': y_pred_list[242], 'N19': y_pred_list[243], 'N15': y_pred_list[244], 'N29': y_pred_list[245], 'N25': y_pred_list[246], 'N39': y_pred_list[247], 'N35': y_pred_list[248], 'N49': y_pred_list[249]})
df10.to_excel(writer,sheet_name='Sheet_name_10')
writer.close()





data_a = []
for i in range(250):
    data_a.append(list(map(float, accuracy[i].split(",")[:250])))
len(data_a)
data_b = []

#for i in range(len(data_a)):
 #   temp = data_a[i]
  #  temp.append(statistics.mean(temp))
   # temp.append(statistics.stdev(temp))
    #data_b.append(temp)

df11 = pd.DataFrame(data_a)#, 'N5': data_b[4], 'N6': data_b[5], 'N7': data_b[6], 'N8': data_b[7], 'N9': data_b[8], 'N10': data_b[9], 'N11': data_b[10], 'N12': data_b[11], 'N13': data_b[12], 'N14': data_b[13], 'N15': data_b[14], 'N16': data_b[15], 'N17': data_b[16], 'N18': data_b[17], 'N19': data_b[18], 'N20': data_b[19], 'N21': data_b[20], 'N22': data_b[21], 'N23': data_b[22], 'N24': data_b[23], 'N25': data_b[24]})
writer = ExcelWriter("performance_b_a.xlsx")
df11.to_excel(writer, 'Sheet1', index=False)
writer.save()


data_b = []
for i in range(250):
    data_b.append(list(map(float, f_score[i].split(",")[:250])))

#data_b = []

#for i in range(len(data_a)):
 #   temp = data_a[i]
  #  temp.append(statistics.mean(temp))
   # temp.append(statistics.stdev(temp))
    #data_b.append(temp)
df11 = pd.DataFrame(data_b)#, 'N3': data_b[2], 'N4': data_b[3], 'N5': data_b[4], 'N6': data_b[5], 'N7': data_b[6], 'N8': data_b[7], 'N9': data_b[8], 'N10': data_b[9], 'N11': data_b[10], 'N12': data_b[11], 'N13': data_b[12], 'N14': data_b[13], 'N15': data_b[14], 'N16': data_b[15], 'N17': data_b[16], 'N18': data_b[17], 'N19': data_b[18], 'N20': data_b[19], 'N21': data_b[20], 'N22': data_b[21], 'N23': data_b[22], 'N24': data_b[23], 'N25': data_b[24]})
writer = ExcelWriter("performance_b_f.xlsx")
df11.to_excel(writer, 'Sheet1', index=False)
writer.save()